# 4. Técnicas Avanzadas de Prompting

## Objetivos de Aprendizaje
- Implementar Tree of Thoughts (ToT) para exploración de soluciones
- Aplicar Self-Consistency para mejorar confiabilidad
- Usar Program-Aided Language Models para cálculos precisos
- Dominar Meta-Prompting y Prompt Chaining

## Introducción a Técnicas Avanzadas

Las técnicas avanzadas de prompting van más allá de los enfoques básicos (zero-shot, few-shot, CoT) para abordar problemas complejos que requieren:
- **Exploración de múltiples caminos** de solución
- **Verificación y consistencia** en las respuestas
- **Integración con herramientas** externas
- **Composición de prompts** para tareas complejas

### Técnicas que Exploraremos:
1. **Tree of Thoughts (ToT)**: Exploración sistemática de alternativas
2. **Self-Consistency**: Múltiples caminos hacia la misma respuesta
3. **Program-Aided Language Models**: LLMs + código ejecutable
4. **Meta-Prompting**: Prompts que generan prompts
5. **Prompt Chaining**: Secuencias de prompts coordinados

In [1]:
# Configuración inicial
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage
import os
import json
import re
from collections import Counter

# Configurar el modelo
llm = ChatOpenAI(
    base_url=os.getenv("OPENAI_BASE_URL"),
    api_key=os.getenv("GITHUB_TOKEN"),
    model="gpt-4.1",
    temperature=0.7  # Balance para exploración y consistencia
)

print("✓ Modelo configurado para técnicas avanzadas")
print("✓ Temperature balanceada para exploración creativa")

✓ Modelo configurado para técnicas avanzadas
✓ Temperature balanceada para exploración creativa


## 1. Tree of Thoughts (ToT)

Tree of Thoughts permite al modelo explorar múltiples caminos de razonamiento de manera sistemática, evaluando y seleccionando las mejores opciones.

In [2]:
# Implementación de Tree of Thoughts
def tree_of_thoughts_ejemplo():
    print("=== TREE OF THOUGHTS (ToT) ===")
    
    # Problema complejo que requiere exploración de alternativas
    problema = """Una startup de foodtech necesita lanzar su app en 3 meses con un presupuesto de 50,000€. 
    Debe decidir entre tres estrategias de desarrollo:
    1. Desarrollo nativo (iOS/Android por separado)
    2. Desarrollo híbrido (React Native/Flutter)
    3. PWA (Progressive Web App)
    
    Factores a considerar: tiempo, costo, rendimiento, experiencia de usuario, escalabilidad futura."""
    
    # Prompt ToT estructurado
    prompt_tot = f"""Usa Tree of Thoughts para resolver este problema de decisión:
    
{problema}
    
PASO 1 - GENERAR RAMAS DE PENSAMIENTO:
Para cada opción (Nativa, Híbrida, PWA), desarrolla un análisis inicial considerando:
- Viabilidad técnica en 3 meses
- Ajuste al presupuesto
- Pros y contras principales
    
PASO 2 - EVALUAR CADA RAMA:
Puntúa cada opción (1-10) en:
- Tiempo de desarrollo
- Costo total
- Calidad UX
- Escalabilidad
- Riesgo técnico
    
PASO 3 - EXPLORAR SUB-RAMAS:
Para las 2 mejores opciones, explora variantes:
- Diferentes frameworks/herramientas
- Estrategias de implementación por fases
- Opciones de tercerización vs. equipo interno
    
PASO 4 - SÍNTESIS Y DECISIÓN:
Basándote en toda la exploración, recomienda la mejor estrategia con plan de implementación.
    
Análisis Tree of Thoughts:"""
    
    try:
        response = llm.invoke([HumanMessage(content=prompt_tot)])
        print("PROBLEMA DE DECISIÓN:")
        print(problema)
        print("\nANÁLISIS TREE OF THOUGHTS:")
        print(response.content)
        
        # Analizar la estructura del ToT
        contenido = response.content.lower()
        pasos_presentes = sum(1 for i in range(1, 5) if f'paso {i}' in contenido)
        opciones_evaluadas = sum(1 for opt in ['nativa', 'híbrida', 'pwa'] if opt in contenido)
        tiene_puntuacion = any(str(i) in contenido for i in range(1, 11))
        
        print(f"\n=== ANÁLISIS DE LA ESTRUCTURA ToT ===")
        print(f"✓ Pasos desarrollados: {pasos_presentes}/4")
        print(f"✓ Opciones evaluadas: {opciones_evaluadas}/3")
        print(f"✓ Incluye puntuaciones: {'Sí' if tiene_puntuacion else 'No'}")
        print(f"✓ Estructura completa: {'Sí' if pasos_presentes >= 3 and opciones_evaluadas >= 2 else 'Parcial'}")
        
    except Exception as e:
        print(f"Error: {e}")

# Ejecutar Tree of Thoughts
tree_of_thoughts_ejemplo()

=== TREE OF THOUGHTS (ToT) ===
PROBLEMA DE DECISIÓN:
Una startup de foodtech necesita lanzar su app en 3 meses con un presupuesto de 50,000€. 
    Debe decidir entre tres estrategias de desarrollo:
    1. Desarrollo nativo (iOS/Android por separado)
    2. Desarrollo híbrido (React Native/Flutter)
    3. PWA (Progressive Web App)

    Factores a considerar: tiempo, costo, rendimiento, experiencia de usuario, escalabilidad futura.

ANÁLISIS TREE OF THOUGHTS:
**Análisis Tree of Thoughts para la decisión de desarrollo de app foodtech**

---

### **PASO 1 - GENERAR RAMAS DE PENSAMIENTO**

#### **Opción 1: Desarrollo Nativo (iOS/Android por separado)**

- **Viabilidad técnica en 3 meses:**  
  *Riesgo alto*. Requiere dos equipos o desarrolladores con experiencia en Swift/Kotlin. Desarrollo paralelo puede complicar coordinación.  
- **Ajuste al presupuesto:**  
  *Difícil*. Dos bases de código, dos equipos: el presupuesto de 50K€ es muy ajustado.  
- **Pros:**  
  - Mejor rendimiento y acces

## 2. Self-Consistency

Self-Consistency mejora la confiabilidad generando múltiples respuestas y seleccionando la más consistente.

In [4]:
# Implementación de Self-Consistency
def self_consistency_ejemplo():
    print("=== SELF-CONSISTENCY ===")
    
    # Problema con múltiples caminos de solución
    problema = """Una empresa de logística necesita optimizar la distribución de 1000 paquetes 
    entre 5 centros de distribución. Los centros tienen capacidades de 150, 200, 250, 200, 200 paquetes respectivamente.
    Los costos de envío por paquete son: Centro A: 2€, Centro B: 1.5€, Centro C: 3€, Centro D: 2.5€, Centro E: 1.8€.
    ¿Cuál es la distribución óptima que minimiza costos?"""
    
    # Generar múltiples respuestas con diferentes enfoques
    enfoques = [
        "Resuelve este problema de optimización priorizando el menor costo por paquete:",
        "Resuelve este problema usando un enfoque de programación lineal simple:",
        "Resuelve este problema considerando tanto costo como capacidad balanceadamente:",
        "Resuelve este problema paso a paso ordenando centros por eficiencia costo-capacidad:"
    ]
    
    respuestas = []
    distribuciones = []
    
    print("PROBLEMA DE OPTIMIZACIÓN:")
    print(problema)
    print("\n=== GENERANDO MÚLTIPLES SOLUCIONES ===")
    
    for i, enfoque in enumerate(enfoques, 1):
        prompt = f"{enfoque}\n\n{problema}\n\nProporciona la distribución específica (número de paquetes por centro) y el costo total."
        
        try:
            response = llm.invoke([HumanMessage(content=prompt)])
            respuestas.append(response.content)
            
            print(f"\nENFOQUE {i}:")
            print(f"Método: {enfoque}")
            print(f"Respuesta: {response.content[:200]}...")
            
            # Extraer números de la respuesta (distribución)
            numeros = re.findall(r'\d+', response.content)
            if len(numeros) >= 5:
                distribucion = [int(x) for x in numeros[:5] if int(x) <= 250]  # Filtrar números razonables
                if len(distribucion) == 5 and sum(distribucion) <= 1100:  # Validación básica
                    distribuciones.append(distribucion)
                    print(f"Distribución extraída: {distribucion}")
            
        except Exception as e:
            print(f"Error en enfoque {i}: {e}")
    
    # Análisis de consistencia
    print(f"\n=== ANÁLISIS DE CONSISTENCIA ===")
    print(f"✓ Respuestas generadas: {len(respuestas)}/4")
    print(f"✓ Distribuciones válidas: {len(distribuciones)}")
    
    if len(distribuciones) >= 2:
        # Calcular distribución promedio
        distribucion_promedio = []
        for i in range(5):
            promedio = sum(dist[i] for dist in distribuciones if i < len(dist)) / len(distribuciones)
            distribucion_promedio.append(round(promedio))
        
        print(f"✓ Distribución consensus: {distribucion_promedio}")
        
        # Verificar consistencia
        variaciones = []
        for i in range(5):
            valores = [dist[i] for dist in distribuciones if i < len(dist)]
            if valores:
                variacion = max(valores) - min(valores)
                variaciones.append(variacion)
        
        variacion_promedio = sum(variaciones) / len(variaciones) if variaciones else 0
        print(f"✓ Variación promedio: {variacion_promedio:.1f} paquetes")
        print(f"✓ Consistencia: {'Alta' if variacion_promedio < 50 else 'Media' if variacion_promedio < 100 else 'Baja'}")
    
    else:
        print("✗ Insuficientes distribuciones válidas para análisis de consistencia")

# Ejecutar Self-Consistency
self_consistency_ejemplo()

=== SELF-CONSISTENCY ===
PROBLEMA DE OPTIMIZACIÓN:
Una empresa de logística necesita optimizar la distribución de 1000 paquetes 
    entre 5 centros de distribución. Los centros tienen capacidades de 150, 200, 250, 200, 200 paquetes respectivamente.
    Los costos de envío por paquete son: Centro A: 2€, Centro B: 1.5€, Centro C: 3€, Centro D: 2.5€, Centro E: 1.8€.
    ¿Cuál es la distribución óptima que minimiza costos?

=== GENERANDO MÚLTIPLES SOLUCIONES ===

ENFOQUE 1:
Método: Resuelve este problema de optimización priorizando el menor costo por paquete:
Respuesta: ### **Problema de optimización**

**Datos:**
- Paquetes totales: **1000**
- Centros y capacidades:
  - A: 150 (2€/paquete)
  - B: 200 (1.5€/paquete)
  - C: 250 (3€/paquete)
  - D: 200 (2.5€/paquete)
 ...

ENFOQUE 2:
Método: Resuelve este problema usando un enfoque de programación lineal simple:
Respuesta: **Planteamiento del problema:**

Sean \( x_1, x_2, x_3, x_4, x_5 \) los paquetes enviados a los centros A, B, C, D, E r

## 3. Program-Aided Language Models (PAL)

PAL combina el razonamiento de LLMs con la precisión de código ejecutable para cálculos exactos.

In [6]:
# Implementación de Program-Aided Language Models
def program_aided_language_model():
    print("=== PROGRAM-AIDED LANGUAGE MODEL (PAL) ===")
    
    # Problema que requiere cálculos precisos
    problema = """Una empresa de suscripciones tiene los siguientes datos mensuales:
    - Usuarios nuevos: 1,250
    - Tasa de conversión a premium: 12.5%
    - Precio premium: 29.99€/mes
    - Churn rate mensual: 8.3%
    - Usuarios premium actuales: 15,420
    - Costo de adquisición por usuario: 18.50€
    
    Calcula: MRR (Monthly Recurring Revenue), crecimiento neto de usuarios premium, 
    y proyección de MRR para los próximos 6 meses."""
    
    # Prompt PAL que genera código ejecutable
    prompt_pal = f"""Resuelve este problema generando código Python ejecutable:
    
{problema}
    
Estructura tu respuesta así:
    
ANÁLISIS:
Explica el problema y los cálculos necesarios.
    
CÓDIGO:
```python
# Código Python para resolver el problema
# Incluye comentarios explicativos
# Define variables claramente
# Realiza cálculos paso a paso
# Muestra resultados formatados
```
    
INTERPRETACIÓN:
Explica los resultados y su significado para el negocio.
    
Solución PAL:"""
    
    try:
        response = llm.invoke([HumanMessage(content=prompt_pal)])
        print("PROBLEMA DE CÁLCULO FINANCIERO:")
        print(problema)
        print("\nSOLUCIÓN PAL:")
        print(response.content)
        
        # Extraer y ejecutar código Python si es posible
        codigo_match = re.search(r'```python\n(.*?)\n```', response.content, re.DOTALL)
        if codigo_match:
            codigo = codigo_match.group(1)
            print("\n=== CÓDIGO EXTRAÍDO ===")
            print(codigo)
            
            # Intentar ejecutar (solo con operaciones seguras)
            if all(palabra not in codigo.lower() for palabra in ['import', 'exec', 'eval', 'open', 'file']):
                try:
                    print("\n=== EJECUCIÓN DEL CÓDIGO ===")
                    # Crear un namespace seguro
                    namespace = {}
                    exec(
                         codigo,
                                 {"__builtins__": {"print": print, "round": round, "range": range}},
                         namespace
                        )

                    """ exec(codigo, {"__builtins__": {"print": print, "round": round}}, namespace) """
                    print("✓ Código ejecutado exitosamente")
                except Exception as e:
                    print(f"✗ Error en ejecución: {e}")
            else:
                print("⚠️ Código contiene imports - no ejecutado por seguridad")
        else:
            print("✗ No se encontró código Python en la respuesta")
        
        # Análisis de la estructura PAL
        contenido = response.content.lower()
        tiene_analisis = 'análisis' in contenido or 'problema' in contenido
        tiene_codigo = '```python' in response.content
        tiene_interpretacion = 'interpretación' in contenido or 'resultado' in contenido
        tiene_comentarios = '#' in response.content
        
        print(f"\n=== ANÁLISIS DE LA ESTRUCTURA PAL ===")
        print(f"✓ Incluye análisis: {'Sí' if tiene_analisis else 'No'}")
        print(f"✓ Incluye código: {'Sí' if tiene_codigo else 'No'}")
        print(f"✓ Incluye interpretación: {'Sí' if tiene_interpretacion else 'No'}")
        print(f"✓ Código comentado: {'Sí' if tiene_comentarios else 'No'}")
        
        estructura_completa = sum([tiene_analisis, tiene_codigo, tiene_interpretacion, tiene_comentarios])
        print(f"✓ Estructura PAL: {'Completa' if estructura_completa >= 3 else 'Parcial'}")
        
    except Exception as e:
        print(f"Error: {e}")

# Ejecutar Program-Aided Language Model
program_aided_language_model()

=== PROGRAM-AIDED LANGUAGE MODEL (PAL) ===
PROBLEMA DE CÁLCULO FINANCIERO:
Una empresa de suscripciones tiene los siguientes datos mensuales:
    - Usuarios nuevos: 1,250
    - Tasa de conversión a premium: 12.5%
    - Precio premium: 29.99€/mes
    - Churn rate mensual: 8.3%
    - Usuarios premium actuales: 15,420
    - Costo de adquisición por usuario: 18.50€

    Calcula: MRR (Monthly Recurring Revenue), crecimiento neto de usuarios premium, 
    y proyección de MRR para los próximos 6 meses.

SOLUCIÓN PAL:
ANÁLISIS:
Debemos calcular tres métricas clave para una empresa de suscripciones:

1. **MRR actual (Monthly Recurring Revenue):**
   - MRR = usuarios premium actuales × precio premium

2. **Crecimiento neto de usuarios premium mensual:**
   - Nuevos usuarios premium = usuarios nuevos × tasa de conversión
   - Usuarios perdidos por churn = usuarios premium actuales × churn rate
   - Crecimiento neto = nuevos usuarios premium - usuarios perdidos

3. **Proyección de MRR para los pró

## 4. Meta-Prompting

Meta-prompting usa el LLM para generar y optimizar prompts para tareas específicas.

In [7]:
# Implementación de Meta-Prompting
def meta_prompting_ejemplo():
    print("=== META-PROMPTING ===")
    
    # Tarea para la cual queremos generar un prompt óptimo
    tarea_objetivo = """Crear un sistema que analice reviews de productos en e-commerce 
    y extraiga información estructurada: sentimiento, aspectos mencionados (precio, calidad, 
    envío, servicio), puntuación de satisfacción, y recomendaciones de mejora para el vendedor."""
    
    # Meta-prompt que genera prompts optimizados
    meta_prompt = f"""Eres un experto en prompt engineering. Tu tarea es crear un prompt óptimo para la siguiente necesidad:
    
TAREA OBJETIVO:
{tarea_objetivo}
    
Diseña un prompt que sea:
1. ESPECÍFICO: Instrucciones claras y detalladas
2. ESTRUCTURADO: Formato de salida bien definido
3. ROBUSTO: Maneje diferentes tipos de reviews
4. EFICIENTE: Obtenga máxima información con mínimo contexto
    
Incluye en tu prompt:
- Rol/contexto para el modelo
- Instrucciones paso a paso
- Formato de salida (preferiblemente JSON)
- Ejemplos si son necesarios
- Manejo de casos edge
    
PROMPT OPTIMIZADO:"""
    
    try:
        response = llm.invoke([HumanMessage(content=meta_prompt)])
        prompt_generado = response.content
        
        print("TAREA PARA LA CUAL GENERAR PROMPT:")
        print(tarea_objetivo)
        print("\nPROMPT GENERADO POR META-PROMPTING:")
        print(prompt_generado)
        
        # Probar el prompt generado con un ejemplo
        review_test = """Producto llegó rápido y bien empaquetado. La calidad es buena por el precio 
        que pagué (25€), aunque esperaba que fuera un poco más resistente. El servicio al cliente 
        respondió rápido cuando tuve una duda. Lo recomendaría para uso ocasional."""
        
        print("\n=== PROBANDO EL PROMPT GENERADO ===")
        print(f"Review de prueba: {review_test}")
        
        # Combinar prompt generado con review de prueba
        prompt_completo = f"{prompt_generado}\n\nReview a analizar: \"{review_test}\""
        
        try:
            test_response = llm.invoke([HumanMessage(content=prompt_completo)])
            print("\nRESULTADO DEL ANÁLISIS:")
            print(test_response.content)
            
            # Evaluar calidad del prompt generado
            resultado = test_response.content
            
            # Verificar elementos esperados
            elementos_esperados = {
                'sentimiento': any(word in resultado.lower() for word in ['positivo', 'negativo', 'neutral', 'sentimiento']),
                'aspectos': any(word in resultado.lower() for word in ['precio', 'calidad', 'envío', 'servicio']),
                'puntuacion': any(char in resultado for char in '0123456789/'),
                'estructura': '{' in resultado and '}' in resultado,  # JSON format
                'recomendaciones': any(word in resultado.lower() for word in ['recomendación', 'mejora', 'sugerencia'])
            }
            
            elementos_presentes = sum(elementos_esperados.values())
            
            print(f"\n=== EVALUACIÓN DEL PROMPT GENERADO ===")
            for elemento, presente in elementos_esperados.items():
                print(f"✓ {elemento.capitalize()}: {'Sí' if presente else 'No'}")
            
            print(f"\n✓ Efectividad del meta-prompt: {elementos_presentes}/5 elementos")
            print(f"✓ Calidad: {'Excelente' if elementos_presentes >= 4 else 'Buena' if elementos_presentes >= 3 else 'Necesita mejoras'}")
            
        except Exception as e:
            print(f"Error probando el prompt: {e}")
        
    except Exception as e:
        print(f"Error en meta-prompting: {e}")

# Ejecutar Meta-Prompting
meta_prompting_ejemplo()

=== META-PROMPTING ===
TAREA PARA LA CUAL GENERAR PROMPT:
Crear un sistema que analice reviews de productos en e-commerce 
    y extraiga información estructurada: sentimiento, aspectos mencionados (precio, calidad, 
    envío, servicio), puntuación de satisfacción, y recomendaciones de mejora para el vendedor.

PROMPT GENERADO POR META-PROMPTING:
PROMPT OPTIMIZADO:

Eres un sistema avanzado de análisis de reviews de productos en e-commerce. Tu tarea es procesar cada review y extraer información estructurada para ayudar al vendedor a mejorar su producto y servicio.

Sigue estos pasos para cada review que recibas:

1. Analiza el texto y determina el sentimiento general: positivo, negativo o neutro.
2. Identifica todos los aspectos mencionados relacionados con el producto o la experiencia de compra. Los principales aspectos son: precio, calidad, envío, servicio. Si se mencionan otros aspectos relevantes, inclúyelos bajo "otros".
3. Asigna una puntuación de satisfacción general del usuari

## 5. Prompt Chaining

Prompt chaining conecta múltiples prompts en secuencia, donde la salida de uno alimenta al siguiente.

In [10]:
def prompt_chaining_ejemplo():
    print("=== PROMPT CHAINING ===")
    
    # Tarea compleja que se beneficia de descomposición
    documento_entrada = """Nuestra empresa SaaS ha experimentado un crecimiento del 150% en usuarios 
    este año, pero los ingresos solo crecieron 80%. El churn rate aumentó de 5% a 12%. 
    Los costos de infraestructura se triplicaron. El equipo de soporte está saturado 
    con 3x más tickets. Los usuarios se quejan de lentitud y bugs. El equipo de desarrollo 
    está al 200% de capacidad. Necesitamos un plan estratégico para los próximos 6 meses."""
    
    print("DOCUMENTO DE ENTRADA:")
    print(documento_entrada)
    print("\n=== INICIANDO CADENA DE PROMPTS ===")
    
    # PASO 1: Análisis de problemas
    prompt_1 = f"""Analiza este reporte empresarial e identifica los problemas principales:
    
{documento_entrada}
    
Extrae y categoriza los problemas en:
1. FINANCIEROS (ingresos, costos, etc.)
2. OPERACIONALES (procesos, capacidad, etc.)
3. TÉCNICOS (rendimiento, bugs, etc.)
4. RECURSOS HUMANOS (capacidad del equipo, etc.)
    
Para cada problema, indica:
- Descripción del problema
- Gravedad (Alta/Media/Baja)
- Impacto en el negocio
    
Análisis de problemas:"""
    
    try:
        response_1 = llm.invoke([HumanMessage(content=prompt_1)])
        analisis_problemas = response_1.content
        
        print("\n📊 PASO 1 - ANÁLISIS DE PROBLEMAS:")
        print(analisis_problemas[:300] + "...")
        
        # PASO 2: Priorización usando resultado del paso 1
        prompt_2 = f"""Basándote en este análisis de problemas, crea una matriz de priorización:
        
ANÁLISIS PREVIO:
{analisis_problemas}
        
Crea una matriz de priorización considerando:
- IMPACTO en el negocio (Alto/Medio/Bajo)
- URGENCIA temporal (Inmediato/Corto plazo/Medio plazo)
- ESFUERZO requerido (Alto/Medio/Bajo)
        
Ordena los problemas por prioridad de atención y explica el criterio.
        
Matriz de priorización:"""
        
        response_2 = llm.invoke([HumanMessage(content=prompt_2)])
        priorizacion = response_2.content
        
        print("\n🎯 PASO 2 - PRIORIZACIÓN:")
        print(priorizacion[:300] + "...")
        
        # PASO 3: Plan de acción usando resultados anteriores
        prompt_3 = f"""Usando este análisis y priorización, desarrolla un plan de acción detallado:
        
ANÁLISIS DE PROBLEMAS:
{analisis_problemas[:500]}...
        
PRIORIZACIÓN:
{priorizacion[:500]}...
        
Desarrolla un plan estratégico de 6 meses que incluya:
        
MES 1-2 (ACCIONES INMEDIATAS):
- Iniciativas críticas
- Recursos necesarios
- Métricas de éxito
        
MES 3-4 (MEJORAS ESTRUCTURALES):
- Proyectos de mediano plazo
- Inversiones requeridas
        
MES 5-6 (OPTIMIZACIÓN):
- Consolidación y refinamiento
- Preparación para crecimiento
        
Plan estratégico:"""
        
        response_3 = llm.invoke([HumanMessage(content=prompt_3)])
        plan_accion = response_3.content
        
        print("\n📋 PASO 3 - PLAN DE ACCIÓN:")
        print(plan_accion[:400] + "...")
        
        # PASO 4: Métricas y KPIs usando toda la información anterior
        # Calcular el número de categorías fuera de la f-string
        num_categorias = len(analisis_problemas.split('\n'))
        
        prompt_4 = f"""Basándote en todo el análisis previo, define métricas y KPIs para monitorear el progreso:
        
CONTEXTO COMPLETO:
- Problemas identificados: {num_categorias} categorías
- Priorización establecida
- Plan de 6 meses definido
        
Define:
        
MÉTRICAS PRINCIPALES (North Star):
- 2-3 métricas clave que reflejen el éxito general
        
KPIs OPERACIONALES:
- Métricas específicas por área (financiera, técnica, operacional)
- Targets cuantitativos para 3 y 6 meses
        
DASHBOARD DE SEGUIMIENTO:
- Frecuencia de medición
- Responsables
- Alertas tempranas
        
Sistema de métricas:"""
        
        response_4 = llm.invoke([HumanMessage(content=prompt_4)])
        metricas = response_4.content
        
        print("\n📈 PASO 4 - MÉTRICAS Y KPIs:")
        print(metricas[:400] + "...")
        
        # Análisis de la efectividad del chaining
        print(f"\n=== ANÁLISIS DEL PROMPT CHAINING ===")
        
        # Verificar progresión lógica
        pasos_completados = 4
        informacion_acumulada = len(analisis_problemas) + len(priorizacion) + len(plan_accion) + len(metricas)
        
        # Verificar coherencia entre pasos
        coherencia_elementos = [
            'problema' in plan_accion.lower(),  # Plan referencia problemas
            'prioridad' in plan_accion.lower(),  # Plan usa priorización
            'mes' in metricas.lower(),  # Métricas alineadas con timeline
            'kpi' in metricas.lower() or 'métrica' in metricas.lower()  # Define métricas específicas
        ]
        
        coherencia = sum(coherencia_elementos)
        
        print(f"✓ Pasos completados: {pasos_completados}/4")
        print(f"✓ Información total generada: ~{informacion_acumulada} caracteres")
        print(f"✓ Coherencia entre pasos: {coherencia}/4 elementos")
        print(f"✓ Efectividad del chaining: {'Excelente' if coherencia >= 3 else 'Buena' if coherencia >= 2 else 'Necesita mejoras'}")
        
        # Beneficios del chaining observados
        print(f"\n✓ Beneficios observados:")
        print(f"  • Descomposición de problema complejo")
        print(f"  • Construcción incremental de solución")
        print(f"  • Cada paso informa al siguiente")
        print(f"  • Resultado final más completo y estructurado")
        
    except Exception as e:
        print(f"Error en prompt chaining: {e}")

# Ejecutar Prompt Chaining
prompt_chaining_ejemplo()

=== PROMPT CHAINING ===
DOCUMENTO DE ENTRADA:
Nuestra empresa SaaS ha experimentado un crecimiento del 150% en usuarios 
    este año, pero los ingresos solo crecieron 80%. El churn rate aumentó de 5% a 12%. 
    Los costos de infraestructura se triplicaron. El equipo de soporte está saturado 
    con 3x más tickets. Los usuarios se quejan de lentitud y bugs. El equipo de desarrollo 
    está al 200% de capacidad. Necesitamos un plan estratégico para los próximos 6 meses.

=== INICIANDO CADENA DE PROMPTS ===

📊 PASO 1 - ANÁLISIS DE PROBLEMAS:
**Análisis de problemas**

---

### 1. FINANCIEROS

**a) Descripción:** El crecimiento de ingresos (80%) es mucho menor que el crecimiento de usuarios (150%).  
**Gravedad:** Alta  
**Impacto:** Margen de ganancia reducido; modelo de monetización ineficiente; riesgo de insostenibilidad si la tendenc...

🎯 PASO 2 - PRIORIZACIÓN:
Claro, aquí tienes la **matriz de priorización** basada en los problemas analizados. Se consideran tres criterios: **Impa

# Comparación de Técnicas Avanzadas

Resumen de cuándo usar cada técnica avanzada.

## Matriz Comparativa

| Técnica | Complejidad | Costo | Tiempo | Precisión | Uso Ideal |
|---------|-------------|-------|---------|-----------|-----------|
| **Tree of Thoughts** | Alta | Muy Alto | Alto | Muy Alta | Decisiones complejas con múltiples caminos posibles |
| **Self-Consistency** | Media | Alto | Alto | Muy Alta | Problemas donde la confiabilidad es crítica |
| **Program-Aided** | Media | Medio | Medio | Muy Alta | Problemas que requieren cálculos precisos |
| **Meta-Prompting** | Media | Medio | Medio | Alta | Cuando necesitas optimizar prompts automáticamente |
| **Prompt Chaining** | Alta | Alto | Alto | Alta | Problemas complejos que se descomponen en pasos |

## 🎯 Guía de Selección

### 🔍 Para EXPLORAR ALTERNATIVAS
- **Usa:** Tree of Thoughts
- **Cuándo:** Cuando hay múltiples caminos válidos

### 🎯 Para MÁXIMA CONFIABILIDAD
- **Usa:** Self-Consistency
- **Cuándo:** Cuando los errores son costosos

### 🧮 Para CÁLCULOS PRECISOS
- **Usa:** Program-Aided
- **Cuándo:** Cuando necesitas exactitud matemática

### ⚙️ Para OPTIMIZAR PROMPTS
- **Usa:** Meta-Prompting
- **Cuándo:** Cuando necesitas automatizar prompt design

### 🔗 Para PROBLEMAS COMPLEJOS
- **Usa:** Prompt Chaining
- **Cuándo:** Cuando el problema se puede descomponer

## 💡 Recomendaciones de Implementación

1. **EMPIEZA SIMPLE:** Prueba técnicas básicas antes de avanzadas
2. **MIDE COSTOS:** Las técnicas avanzadas consumen más tokens
3. **VALIDA RESULTADOS:** Siempre verifica la calidad de salida
4. **COMBINA TÉCNICAS:** Puedes usar múltiples enfoques complementarios
5. **AUTOMATIZA:** Considera scripts para técnicas que usarás frecuentemente

## Ejercicio Integrador

Aplicar múltiples técnicas avanzadas a un problema empresarial complejo.

In [ ]:
# Ejercicio final: Caso integrador
def ejercicio_integrador():
    print("=== EJERCICIO INTEGRADOR: TÉCNICAS AVANZADAS ===")
    
    caso_complejo = """CASO: EXPANSIÓN INTERNACIONAL DE STARTUP
    
    TechFlow, una startup española de software de gestión de proyectos, tiene:
    - 50,000 usuarios activos (70% España, 20% LATAM, 10% resto)
    - ARR de 2.4M€, crecimiento 180% anual
    - Equipo de 25 personas (15 desarrollo, 5 ventas, 5 ops)
    - Funding Serie A de 5M€ recientemente cerrada
    
    DESAFÍO: Decidir estrategia de expansión internacional para los próximos 18 meses.
    
    OPCIONES CONSIDERADAS:
    1. Europa (Francia, Italia, Alemania) - mercado similar, regulación conocida
    2. USA - mercado grande, competencia intensa, regulación compleja
    3. LATAM (México, Colombia, Chile) - afinidad cultural, menor poder adquisitivo
    4. Consolidación España - dominar mercado local antes de expandir
    
    RESTRICCIONES:
    - Presupuesto: 1.5M€ para expansión
    - Tiempo: Decisión debe tomarse en 2 meses
    - Recursos: Solo puede abrir 1-2 mercados simultáneamente
    - Regulación: Debe cumplir GDPR y regulaciones locales"""
    
    print(caso_complejo)
    
    print("\n🎯 EJERCICIO PROPUESTO:")
    print("\nDiseña una estrategia de análisis usando MÚLTIPLES técnicas avanzadas:")
    
    estrategia_sugerida = """
    PASO 1: META-PROMPTING
    - Genera prompts optimizados para analizar cada mercado objetivo
    
    PASO 2: PROGRAM-AIDED ANALYSIS  
    - Calcula métricas financieras: TAM, CAC, LTV por mercado
    - Proyecciones de revenue y costos de entrada
    
    PASO 3: TREE OF THOUGHTS
    - Explora sistemáticamente pros/contras de cada opción
    - Considera factores: mercado, competencia, recursos, timing
    
    PASO 4: SELF-CONSISTENCY
    - Genera múltiples recomendaciones con diferentes enfoques
    - Identifica consenso o áreas de incertidumbre
    
    PASO 5: PROMPT CHAINING
    - Integra todos los análisis anteriores
    - Desarrolla plan de implementación detallado
    - Define timeline, milestones y métricas de éxito
    """
    
    print(estrategia_sugerida)
    
    print("\n💡 TU IMPLEMENTACIÓN:")
    print("\n1. Elige 2-3 técnicas que consideres más apropiadas para este caso")
    print("2. Diseña los prompts específicos para cada técnica")
    print("3. Ejecuta tu análisis")
    print("4. Compara resultados entre técnicas")
    print("5. Proporciona recomendación final integrada")
    
    # Template para implementación del estudiante
    template_implementacion = """
    # IMPLEMENTACIÓN DEL ESTUDIANTE
    
    ## TÉCNICAS SELECCIONADAS:
    1. [Técnica 1]: [Justificación]
    2. [Técnica 2]: [Justificación] 
    3. [Técnica 3]: [Justificación]
    
    ## PROMPTS DISEÑADOS:
    
    ### Prompt para [Técnica 1]:
    [Tu prompt aquí]
    
    ### Prompt para [Técnica 2]:
    [Tu prompt aquí]
    
    ## ANÁLISIS COMPARATIVO:
    [Compara resultados entre técnicas]
    
    ## RECOMENDACIÓN FINAL:
    [Tu recomendación integrada]
    """
    
    print(template_implementacion)
    
    # Criterios de evaluación
    print("\n📋 CRITERIOS DE EVALUACIÓN:")
    criterios = [
        "✓ Selección apropiada de técnicas para el problema",
        "✓ Calidad y estructura de los prompts diseñados", 
        "✓ Profundidad del análisis realizado",
        "✓ Integración efectiva de múltiples perspectivas",
        "✓ Practicidad y viabilidad de la recomendación final"
    ]
    
    for criterio in criterios:
        print(f"   {criterio}")
    
    print("\n🏆 OBJETIVO: Demostrar dominio de técnicas avanzadas aplicadas a casos reales")

# Ejecutar ejercicio integrador
ejercicio_integrador()

## Conceptos Clave Aprendidos

1. **Tree of Thoughts** permite exploración sistemática de alternativas
2. **Self-Consistency** mejora confiabilidad mediante múltiples caminos
3. **Program-Aided Language Models** combinan razonamiento LLM + precisión código
4. **Meta-Prompting** automatiza la optimización de prompts
5. **Prompt Chaining** descompone problemas complejos en pasos manejables

## Cuándo Usar Cada Técnica

### 🎯 Matriz de Decisión:
- **Alta precisión requerida** → Self-Consistency + PAL
- **Múltiples alternativas** → Tree of Thoughts
- **Problema complejo** → Prompt Chaining
- **Automatización** → Meta-Prompting
- **Cálculos críticos** → Program-Aided

### ⚖️ Trade-offs:
- **Precisión vs. Costo**: Técnicas avanzadas son más precisas pero más costosas
- **Complejidad vs. Control**: Mayor control requiere mayor complejidad de implementación
- **Tiempo vs. Calidad**: Mejores resultados requieren más tiempo de procesamiento

### Para Practicar:
1. Implementa cada técnica en tu dominio específico
2. Combina múltiples técnicas para problemas complejos
3. Mide y compara efectividad vs. costo
4. Desarrolla templates reutilizables para tu uso frecuente